# Engenharia de Machine Learning - XP Educação - Desafio 3 - nov/2022


## Objetivos
Exercitar os seguintes conceitos trabalhados no Módulo:
- Neste desafio, vamos trabalhar os conceitos vistos sobre métricas de desempenho, técnicas
de validação e sintonia de hiperparâmetros.
- Também vamos olhar para o fluxo completo de seleção de um modelo de aprendizado.

Dataset: https://www.openml.org/search?type=data&sort=runs&id=1480&status=active

## Carregando as bibliotecas e o dataset

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, make_scorer, roc_auc_score
from sklearn import metrics

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from scipy.stats import randint


In [36]:
# carregando o dataset
df = pd.read_csv("ilpd.csv", index_col='id') # Temos uma coluna 'id' e vamos usá-la como index do dataframe
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,Class
id,,,,,,,,,,,
1,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1
2,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
3,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
4,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1
5,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1


In [37]:
df.tail()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,Class
id,,,,,,,,,,,
579,60,Male,0.5,0.1,500,20,34,5.9,1.6,0.37,2
580,40,Male,0.6,0.1,98,35,31,6.0,3.2,1.10,1
581,52,Male,0.8,0.2,245,48,49,6.4,3.2,1.00,1
582,31,Male,1.3,0.5,184,29,32,6.8,3.4,1.00,1
583,38,Male,1.0,0.3,216,21,24,7.3,4.4,1.50,2


## 1) O objetivo dessa base é avaliar se uma pessoa vai ser um paciente hepático ou não. A feature que determina a classe nessa base é a “V7”.


### Resposta: Falso, a feature V7 é de Sgot Aspartate Aminotransferase segundo a descrição no site e a feature Class é a que determina a classe

## 2) Sobre as informações de número de features e amostras, marque a alternativa CORRETA. 


In [38]:
df.shape

(583, 11)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 583 entries, 1 to 583
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   V1      583 non-null    int64  
 1   V2      583 non-null    object 
 2   V3      583 non-null    float64
 3   V4      583 non-null    float64
 4   V5      583 non-null    int64  
 5   V6      583 non-null    int64  
 6   V7      583 non-null    int64  
 7   V8      583 non-null    float64
 8   V9      583 non-null    float64
 9   V10     583 non-null    float64
 10  Class   583 non-null    int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 54.7+ KB


### Resposta: 583 amostras e 11 features incluindo o atributo classe

## 3) Essa base conta com 3 atributos categóricos. Precisamos fazer o pré processamento desses atributos para que todos sejam numéricos.


#### Temos 2 atributos categóricos: V2 e Class apesar de ser numérico 

### Resposta: Falso, apenas 2 atributos categóricos

## 4) A base não tem dados faltantes. 


In [40]:
df.isnull().sum()

V1       0
V2       0
V3       0
V4       0
V5       0
V6       0
V7       0
V8       0
V9       0
V10      0
Class    0
dtype: int64

In [41]:
df.isna().sum()

V1       0
V2       0
V3       0
V4       0
V5       0
V6       0
V7       0
V8       0
V9       0
V10      0
Class    0
dtype: int64

### Resposta: Verdadeiro

## 5) Sobre o best_score_ encontrado durante a otimização do SVC, marque a alternativa CORRETA. 


In [42]:
# Analisando a variável V2
df['V2'].unique()

array(['Female', 'Male'], dtype=object)

In [43]:
#criando um dicionario de dados para o mapeamento
name_to_class = {'Female': 0, 'Male': 1}

#substituindo os valores categóricos pelo mapeamento
# Female: 0 , Male = 1
df['V2'] = df['V2'].map(name_to_class)

#check
df.head(5)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,Class
id,,,,,,,,,,,
1,65,0,0.7,0.1,187,16,18,6.8,3.3,0.90,1
2,62,1,10.9,5.5,699,64,100,7.5,3.2,0.74,1
3,62,1,7.3,4.1,490,60,68,7.0,3.3,0.89,1
4,58,1,1.0,0.4,182,14,20,6.8,3.4,1.00,1
5,72,1,3.9,2.0,195,27,59,7.3,2.4,0.40,1


In [44]:
# Separando a variável alvo
x = df.drop('Class', axis = 1)
y = df['Class']

##### Separando os dados em treinamento e teste

In [45]:
# aplicando a funcao train_test_split para separar os conjuntos de treino e 
# teste segundo uma porcentagem de separação definida. 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 5762)

### Random Search

In [46]:
#definindo o tipo de validacao cruzada e o numero de folds
cv_strat = StratifiedKFold(n_splits = 10)

#definindo a estrategia de score a partir da metrica f1
f1 = make_scorer(f1_score)

#definindo hiperparâmetros
distributions = dict(kernel = ['sigmoid', 'poly', 'rbf'], 
                     C = uniform(loc=1, scale=10))

#instânciando meu classificador
classifier = SVC()

#instanciando e modelando o grid search com os hiperparametros e a validação definidas.
random_cv = RandomizedSearchCV(classifier, distributions, cv = cv_strat, scoring = f1, random_state = 5762, n_iter = 5)
random_cv.fit(x, y)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
                   estimator=SVC(), n_iter=5,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000025443DDAB00>,
                                        'kernel': ['sigmoid', 'poly', 'rbf']},
                   random_state=5762, scoring=make_scorer(f1_score))

In [47]:
#vamos olhar para todos os resultados encontrados!
print('Resumo de todos os resultados encontrados:\n\n', random_cv.cv_results_)

Resumo de todos os resultados encontrados:

 {'mean_fit_time': array([0.49055359, 0.34823775, 0.01406317, 0.2690455 , 0.01562612]), 'std_fit_time': array([2.34117809e-01, 1.97063408e-01, 4.68772279e-03, 1.76536595e-01,
       1.57807167e-06]), 'mean_score_time': array([0.00236268, 0.00425673, 0.00468798, 0.00216284, 0.00312531]), 'std_score_time': array([0.00469419, 0.00561861, 0.007161  , 0.00464151, 0.00625062]), 'param_C': masked_array(data=[9.070941686647695, 3.996689355015553, 2.96908678529896,
                   3.038065179879575, 6.043494590721352],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'param_kernel': masked_array(data=['poly', 'poly', 'rbf', 'poly', 'rbf'],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'C': 9.070941686647695, 'kernel': 'poly'}, {'C': 3.996689355015553, 'kernel': 'poly'}, {'C': 2.96908678529896, 'kernel': 'rbf'}, {'C': 3.0380

In [48]:
#vamos olhar para os melhores resultados encontrados pelo Grid Search
print('Melhor resultado f1:', random_cv.best_score_)
print('\n\nMelhor configuração de hiperparâmetros:', random_cv.best_params_)
print( '\n\nConfigurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: \n', random_cv.best_estimator_)

Melhor resultado f1: 0.832818081808181


Melhor configuração de hiperparâmetros: {'C': 3.996689355015553, 'kernel': 'poly'}


Configurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: 
 SVC(C=3.996689355015553, kernel='poly')


### Resposta: O valor da f1 encontrado foi aproximadamente 0.83

## 6) O best_params_ encontrado durante a otimização do SVC indica que a melhor combinação de parâmetros usa um kernel RBF. 


In [51]:
print('\n\nMelhor configuração de hiperparâmetros:', random_cv.best_params_)



Melhor configuração de hiperparâmetros: {'C': 3.996689355015553, 'kernel': 'poly'}


### Resposta: Falso, o melhor é o kernel Polinomial

## 7) O best_params_ encontrado durante a otimização do SVC indica que C encontrado é aproximadamente 5. 


### Resposta: Falso, é aproximadamente 4

## 8) A melhor combinação de best_params_ encontrado durante a otimização do SVC é: 


### Resposta: Kernel Poly com C = 3.996

## 9) Sobre o best_score_ encontrado durante a otimização do Random Forest, marque a alternativa CORRETA. 


In [49]:
#definindo o tipo de validacao cruzada e o numero de folds
cv_strat = StratifiedKFold(n_splits = 10)

#definindo a estrategia de score a partir da metrica f1
f1 = make_scorer(f1_score)

#definindo hiperparâmetros
distributions2 = dict(n_estimators = randint(10, 1000),
                      bootstrap = [True, False],
                      criterion = ['gini', 'entropy'])

#instânciando meu classificador
classifier2 = RandomForestClassifier(random_state = 5762)

#instanciando e modelando o grid search com os hiperparametros e a validação definidas.
random_cv2 = RandomizedSearchCV(classifier2, distributions2, cv = cv_strat, scoring = f1, random_state = 5762, n_iter = 5)
random_cv2.fit(x, y)

RandomizedSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=False),
                   estimator=RandomForestClassifier(random_state=5762),
                   n_iter=5,
                   param_distributions={'bootstrap': [True, False],
                                        'criterion': ['gini', 'entropy'],
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000025443DDA680>},
                   random_state=5762, scoring=make_scorer(f1_score))

In [50]:
#vamos olhar para os melhores resultados encontrados pelo Grid Search
print('Melhor resultado f1:', random_cv2.best_score_)
print('\n\nMelhor configuração de hiperparâmetros:', random_cv2.best_params_)
print( '\n\nConfigurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: \n', random_cv2.best_estimator_)

Melhor resultado f1: 0.7984322518413547


Melhor configuração de hiperparâmetros: {'bootstrap': True, 'criterion': 'gini', 'n_estimators': 488}


Configurações de todos os hiperparâmetros do melhor estimado encontrado pelo GridSearch: 
 RandomForestClassifier(n_estimators=488, random_state=5762)


### Resposta: O valor da f1 encontrado foi aproximadamente 0.80

## 10) O best_params_ encontrado durante a otimização do Random Forest indica que a melhor combinação de parâmetros usa Boostrap True. 


### Resposta: Verdadeiro

## 11) O best_params_ encontrado durante a otimização do Random Forest indica que a melhor combinação de parâmetros usa Criterion Entropy. 


### Resposta: Falso, o melhor critério é o "Gini"

## 12) O best_params_ encontrado durante a otimização do Random Forest indica que a melhor combinação de parâmetros conta com quantos estimadores? 

In [53]:
print('\n\nMelhor configuração de hiperparâmetros:', random_cv2.best_params_)



Melhor configuração de hiperparâmetros: {'bootstrap': True, 'criterion': 'gini', 'n_estimators': 488}


### Resposta: n_estimators: 488

## 13) Suponha que nossas duas opções para solução do problema sejam o SVC e o Random Forest que foram modelados anteriormente. Além disso, não é possível rodar o experimento novamente com outros parâmetros. Marque a alternativa que justifica de forma CORRETA a escolha do modelo mais adequado.

In [54]:
print('Melhor resultado f1 para SVC:', random_cv.best_score_)
print('Melhor resultado f1 para Random Forest:', random_cv2.best_score_)


Melhor resultado f1 para SVC: 0.832818081808181
Melhor resultado f1 para Random Forest: 0.7984322518413547


### Resposta: O SVC é o modelo que deveria ser escolhido já que ele apresenta uma f1 maior

## 14)  Como os dois modelos apresentaram resultados semelhantes de f1, podemos entender que a capacidade de generalização dos modelos deve ser semelhante na prática. Nesse caso, não faz sentido a análise de outras métricas para validação dos modelos. 


### Resposta: Falso

## 15) Os modelos treinados apresentam resultados de f1 semelhantes por se tratarem de modelos com estrutura de implementação semelhantes, são ambos modelos de árvores de decisão. Nesse caso, sempre que treinarmos um modelo de árvore, a tendência é que o resultado de f1 seja próximo aos encontrados nesse desafio

### Resposta: Falso